In [1]:
%load_ext autoreload
%autoreload 2

import odc.stac
import pandas as pd
import pystac_client

from pyTMD.compute import tide_elevations
import pandas as pd
import numpy as np


GAUGE_X = 122.2183
GAUGE_Y = -18.0008
ENSEMBLE_MODELS = ["EOT20", "HAMTIDE11"]  # simplified for tests

## Load fixtures

In [3]:
def load_satellite_ds():
    """
    Load a sample timeseries of Landsat 8 data using odc-stac
    """
    # Connect to stac catalogue
    catalog = pystac_client.Client.open("https://explorer.dea.ga.gov.au/stac")

    # Set cloud defaults
    odc.stac.configure_rio(
        cloud_defaults=True,
        aws={"aws_unsigned": True},
    )

    # Build a query with the parameters above
    buffer = 0.08
    # buffer = 0.5
    bbox = [GAUGE_X - buffer, GAUGE_Y - buffer, GAUGE_X + buffer, GAUGE_Y + buffer]
    query = catalog.search(
        bbox=bbox,
        collections=["ga_ls8c_ard_3"],
        datetime="2020-01/2020-02",
    )

    # Search the STAC catalog for all items matching the query
    ds = odc.stac.load(
        list(query.items()),
        bands=["nbart_red"],
        crs="epsg:3577",
        resolution=30,
        groupby="solar_day",
        bbox=bbox,
        fail_on_error=False,
        chunks={},
    )

    return ds

satellite_ds = load_satellite_ds()

def load_measured_tides_ds():
    """
    Load measured sea level data from the Broome ABSLMP tidal station:
    http://www.bom.gov.au/oceanography/projects/abslmp/data/data.shtml
    """
    # Metadata for Broome ABSLMP tidal station:
    # http://www.bom.gov.au/oceanography/projects/abslmp/data/data.shtml
    ahd_offset = -5.322

    # Load measured tides from ABSLMP tide gauge data
    measured_tides_df = pd.read_csv(
        "../tests/data/IDO71013_2020.csv",
        index_col=0,
        parse_dates=True,
        na_values=-9999,
    )[["Sea Level"]]

    # Update index and column names
    measured_tides_df.index.name = "time"
    measured_tides_df.columns = ["tide_height"]

    # Apply station AHD offset
    measured_tides_df += ahd_offset

    # Return as xarray dataset
    return measured_tides_df.to_xarray()

satellite_ds = load_satellite_ds()
measured_tides_ds = load_measured_tides_ds()

## Testing pyTMD

In [ ]:
from eo_tides import model_tides

x, y, crs, method, model = GAUGE_X, GAUGE_Y, "EPSG:4326", "spline", "EOT20"
x, y, crs, method, model = GAUGE_X, GAUGE_Y, "EPSG:4326", "bilinear", "EOT20"
x, y, crs, method, model = -1034913, -1961916, "EPSG:3577", "bilinear", "EOT20"


# Run EOT20 tidal model for locations and timesteps in tide gauge data
modelled_tides_df = model_tides(
    x=[x],
    y=[y],
    time=measured_tides_ds.time,
    crs=crs,
    method=method,
    directory="../tests/data/tide_models",
)

# Run equivalent pyTMD code to verify same results
pytmd_tides = tide_elevations(
        x=x, 
        y=y, 
        delta_time=measured_tides_ds.time,
        DIRECTORY="../tests/data/tide_models",
        MODEL="EOT20",
        EPSG=int(crs[-4:]),
        TIME="datetime",
        EXTRAPOLATE=True,
        CUTOFF=np.inf,
        METHOD=method,
        # CORRECTIONS: str | None = None,
        # INFER_MINOR: bool = True,
        # MINOR_CONSTITUENTS: list | None = None,
        # APPLY_FLEXURE: bool = False,
        # FILL_VALUE: float = np.nan
        )

np.allclose(modelled_tides_df.tide_height.values, pytmd_tides.data)

### Error for out of bounds

In [16]:
from eo_tides import model_tides

x, y = 180, -50


# Run EOT20 tidal model for locations and timesteps in tide gauge data
modelled_tides_df = model_tides(
    x=[x],
    y=[y],
    model=["EOT20", "GOT5.5"],
    time=measured_tides_ds.time,
    directory="../tests/data/tide_models",
)

Modelling tides using EOT20, GOT5.5 in parallel


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]


Exception: The EOT20 tide model constituent files do not cover the requested analysis extent.
This can occur if you are using clipped model files to improve run times.
Consider using model files that cover your entire analysis area, or set `crop=False`
to reduce the extent of tide model constituent files that is loaded.

In [35]:
from eo_tides import list_models
list_models(directory="")

──────────────────────────────────────────────────────────────
 󠀠🌊  | Model                | Expected path                   
──────────────────────────────────────────────────────────────
 ❌  │ AODTM-5              │ aodtm5_tmd                      
 ❌  │ AOTIM-5              │ aotim5_tmd                      
 ❌  │ AOTIM-5-2018         │ Arc5km2018                      
 ❌  │ Arc2kmTM             │ Arc2kmTM                        
 ❌  │ CATS0201             │ cats0201_tmd                    
 ❌  │ CATS2008             │ CATS2008                        


 ❌  │ CATS2008-v2023       │ CATS2008_v2023                  
 ❌  │ CATS2008_load        │ CATS2008a_SPOTL_Load            
 ❌  │ EOT20                │ EOT20/ocean_tides               
 ❌  │ EOT20_load           │ EOT20/load_tides                
 ❌  │ FES2012              │ fes2012/data                    
 ❌  │ FES2014              │ fes2014/ocean_tide              
 ❌  │ FES2014_extrapolated │ fes2014/ocean_tide_extrapolated 
 ❌  │ FES2014_load         │ fes2014/load_tide               
 ❌  │ FES2022              │ fes2022b/ocean_tide             
 ❌  │ FES2022_extrapolated │ fes2022b/ocean_tide_extrapolated
 ❌  │ FES2022_load         │ fes2022b/load_tide              
 ❌  │ GOT4.10              │ GOT4.10c/grids_oceantide        
 ❌  │ GOT4.10_load         │ GOT4.10c/grids_loadtide         
 ❌  │ GOT4.7               │ GOT4.7/grids_oceantide          
 ❌  │ GOT4.7_load          │ GOT4.7/grids_loadtide           
 ❌  │ GOT4.8               │ got4.8/grids_oceantide          
 ❌  │ G

/workspaces/eo-tides/eo_tides/model.py:164: UserWarning: No valid tide models are available in `.`.
Are you sure you have provided the correct `directory` path, or set the 
`EO_TIDES_TIDE_MODELS` environment variable to point to the location of your 
tide model directory?
  warnings.warn(warning_msg, UserWarning)


([],
 ['AODTM-5',
  'AOTIM-5',
  'AOTIM-5-2018',
  'Arc2kmTM',
  'CATS0201',
  'CATS2008',
  'CATS2008-v2023',
  'CATS2008_load',
  'EOT20',
  'EOT20_load',
  'FES2012',
  'FES2014',
  'FES2014_extrapolated',
  'FES2014_load',
  'FES2022',
  'FES2022_extrapolated',
  'FES2022_load',
  'GOT4.10',
  'GOT4.10_load',
  'GOT4.7',
  'GOT4.7_load',
  'GOT4.8',
  'GOT4.8_load',
  'GOT5.5',
  'GOT5.5D',
  'GOT5.5D_extrapolated',
  'GOT5.5_extrapolated',
  'GOT5.5_load',
  'GOT5.6',
  'GOT5.6_extrapolated',
  'Gr1km-v2',
  'Gr1kmTM',
  'HAMTIDE11',
  'TPXO10-atlas-v2',
  'TPXO10-atlas-v2-nc',
  'TPXO7.2',
  'TPXO7.2_load',
  'TPXO8-atlas',
  'TPXO8-atlas-nc',
  'TPXO9-atlas',
  'TPXO9-atlas-nc',
  'TPXO9-atlas-v2',
  'TPXO9-atlas-v2-nc',
  'TPXO9-atlas-v3',
  'TPXO9-atlas-v3-nc',
  'TPXO9-atlas-v4',
  'TPXO9-atlas-v4-nc',
  'TPXO9-atlas-v5',
  'TPXO9-atlas-v5-nc',
  'TPXO9.1'])

### Modelling ebb and flow tidal phases
The `tag_tides` function also allows us to determine whether each satellite observation was taken while the tide was rising/incoming (flow tide) or falling/outgoing (ebb tide) by setting `ebb_flow=True`. This is achieved by comparing tide heights 15 minutes before and after the observed satellite observation.

Ebb and flow data can provide valuable contextual information for interpreting satellite imagery, particularly in tidal flat or mangrove forest environments where water may remain in the landscape for considerable time after the tidal peak.

Once you run the cell below, our data will now also contain a new `ebb_flow` variable under **Data variables**:

In [ ]:
# Model tide heights
ds = tag_tides(
    ds, 
    ebb_flow=True,     
    directory="../../tests/data/tide_models",
)

# Print output data
print(ds)

We now have data giving us the both the tide height and tidal phase ("ebb" or "flow") for every satellite image:

In [ ]:
ds[["time", "tide_height", "ebb_flow"]].drop_vars("spatial_ref").to_dataframe().head()

We could for example use this data to filter our observations to keep ebbing phase observations only:

In [ ]:
ds_ebb = ds.where(ds.ebb_flow == "Ebb", drop=True)
print(ds_ebb)

## Pixel biases

In [ ]:
import odc.stac
import pystac_client
import planetary_computer

# Connect to STAC catalog
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

# Set cloud access defaults
odc.stac.configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
)

# Build a query and search the STAC catalog for all matching items
bbox = [122.160, -18.05, 122.260, -17.95]
query = catalog.search(
    bbox=bbox,
    collections=["sentinel-2-l2a"],
    datetime="2021/2023",
)

# Load data into xarray format
ds_s2 = odc.stac.load(
    items=list(query.items()),
    bands=["red"],
    crs="utm",
    resolution=30,
    groupby="solar_day",
    bbox=bbox,
    fail_on_error=False,
    chunks={},
)

print(ds_s2)

Creating reduced resolution 5000 x 5000 metre tide modelling array
Modelling tides using EOT20, GOT5.5 in parallel


100%|██████████| 10/10 [00:00<00:00, 10.10it/s]


Computing tide quantiles
Returning low resolution tide array
Creating reduced resolution 5000 x 5000 metre tide modelling array
Modelling tides using EOT20, GOT5.5 in parallel


100%|██████████| 10/10 [00:02<00:00,  4.76it/s]


Computing tide quantiles
Returning low resolution tide array
<xarray.DataArray 'tide_height' (tide_model: 2, y: 9, x: 10)> Size: 720B
array([[[-1.7344081, -1.760125 , -1.7868625, -1.8580176,        nan,
                nan,        nan,        nan,        nan,        nan],
        [-1.7684059, -1.8007368, -1.8340894, -1.8580176, -1.8580176,
                nan,        nan,        nan,        nan,        nan],
        [-1.806048 , -1.8471183, -1.8895663, -1.8580176, -1.8580176,
         -2.1748762, -2.1748762,        nan,        nan,        nan],
        [-1.846053 , -1.8960319, -1.9473902, -2.0383308, -2.1748762,
         -2.1748762, -2.1748762, -2.1748762,        nan,        nan],
        [-1.8846477, -1.9435406, -2.0038147, -2.0623674, -2.1157305,
         -2.16796  , -2.1748762, -2.1748762,        nan,        nan],
        [-1.9184506, -1.9783273, -2.038216 , -2.0931957, -2.139294 ,
         -2.1842573, -2.1748762,        nan,        nan,        nan],
        [-1.9518493, -2.0111885,

In [22]:
list(stats_ds.data_vars.keys())

['hat',
 'hot',
 'lat',
 'lot',
 'otr',
 'tr',
 'spread',
 'offset_low',
 'offset_high']

In [73]:
from eo_tides.stats import pixel_stats

models = ["EOT20"]
resample = True

stats_ds = pixel_stats(
    ds=satellite_ds,
    model=models,
    resample=resample,
    directory="../tests/data/tide_models",
)

# Verify dims are correct
assert stats_ds.odc.spatial_dims == satellite_ds.odc.spatial_dims

# Verify vars are as expected
expected_vars = ['hat',  'hot',  'lat',  'lot',  'otr',  'tr',  'spread',  'offset_low',  'offset_high']
assert set(expected_vars) == set(stats_ds.data_vars)

# Verify tide models are correct
assert all(stats_ds["tide_model"].values == models)
if len(models) > 1:
    assert "tide_model" in stats_ds.dims

# If resample, assert that statistics have the same shape and dims
# as `satellite_ds`
if resample:
    assert satellite_ds.odc.geobox.shape == stats_ds.odc.geobox.shape



Creating reduced resolution 5000 x 5000 metre tide modelling array


Modelling tides using EOT20 in parallel


100%|██████████| 5/5 [00:00<00:00,  7.36it/s]


Computing tide quantiles
Returning low resolution tide array
Creating reduced resolution 5000 x 5000 metre tide modelling array
Modelling tides using EOT20 in parallel


100%|██████████| 5/5 [00:01<00:00,  3.95it/s]


Computing tide quantiles
Returning low resolution tide array


In [75]:
# Verify values are roughly expected
assert np.allclose(stats_ds.offset_high.mean().item, 0.30, atol=0.02)
assert np.allclose(stats_ds.offset_low.mean().item, 0.27, atol=0.02)
assert np.allclose(stats_ds.spread.mean().item, 0.43, atol=0.02)

TypeError: unsupported operand type(s) for -: 'method' and 'float'

In [77]:
stats_ds.offset_high.mean().item()

0.3040720224380493

True

In [69]:
stats_ds.spread.mean()

<xarray.DataArray 'spread' ()> Size: 4B
array(0.43034247, dtype=float32)
Coordinates:
    spatial_ref  int32 4B 3577

In [43]:
stats_ds["tide_model"].values.tolist()

[np.str_('EOT20'), np.str_('GOT5.5')]

In [41]:
stats_ds["tide_model"].values.tolist()

'EOT20'

In [25]:
set(['hat',  'hot',  'lat',  'lot',  'otr',  'tr',  'spread',  'offset_low',  'offset_high'])

{'hat',
 'hot',
 'lat',
 'lot',
 'offset_high',
 'offset_low',
 'otr',
 'spread',
 'tr'}

In [24]:
set(stats_ds.data_vars)

{'hat',
 'hot',
 'lat',
 'lot',
 'offset_high',
 'offset_low',
 'otr',
 'spread',
 'tr'}

In [14]:
from eo_tides import pixel_tides

pixel_tides(
    ds=satellite_ds,
    model=["EOT20", "GOT5.5"],
    directory="../tests/data/tide_models",
    )

Creating reduced resolution 5000 x 5000 metre tide modelling array


Modelling tides using EOT20, GOT5.5 in parallel


100%|██████████| 10/10 [00:01<00:00,  9.60it/s]


Reprojecting tides into original resolution


<xarray.DataArray 'tide_height' (tide_model: 2, time: 7, y: 633, x: 609)> Size: 22MB
array([[[[-1.3144574 , -1.3144041 , -1.314351  , ..., -1.3897811 ,
          -1.3897811 , -1.3897811 ],
         [-1.3144894 , -1.3144355 , -1.3143817 , ..., -1.3897811 ,
          -1.3897811 , -1.3897811 ],
         [-1.3145566 , -1.3145025 , -1.3144484 , ..., -1.3897811 ,
          -1.3897811 , -1.3897811 ],
         ...,
         [-1.379683  , -1.379747  , -1.3798112 , ..., -1.3997254 ,
          -1.3997254 , -1.3997254 ],
         [-1.3797381 , -1.3798022 , -1.3798662 , ..., -1.399803  ,
          -1.399803  , -1.399803  ],
         [-1.3797933 , -1.3798573 , -1.3799213 , ..., -1.3998806 ,
          -1.3998806 , -1.3998806 ]],

        [[ 0.6604065 ,  0.66045976,  0.6605131 , ...,  0.542365  ,
           0.542365  ,  0.542365  ],
         [ 0.6603817 ,  0.6604355 ,  0.66048926, ...,  0.542365  ,
           0.542365  ,  0.542365  ],
         [ 0.6603187 ,  0.6603726 ,  0.6604266 , ...,  0.542365  ,
           0.542365  ,  0.542365  ],
...
         [-2.047474  , -2.047807  , -2.0481403 , ..., -2.1709404 ,
          -2.1710033 , -2.171066  ],
         [-2.0476592 , -2.0479925 , -2.0483255 , ..., -2.171011  ,
          -2.1710737 , -2.1711361 ],
         [-2.0478444 , -2.0481777 , -2.0485108 , ..., -2.1710815 ,
          -2.171144  , -2.1712065 ]],

        [[ 1.6737119 ,  1.6737957 ,  1.6738795 , ...,  1.713383  ,
           1.7138532 ,  1.7143235 ],
         [ 1.6738306 ,  1.6739147 ,  1.6739987 , ...,  1.7135384 ,
           1.7140115 ,  1.7144847 ],
         [ 1.6739459 ,  1.6740301 ,  1.6741141 , ...,  1.7137088 ,
           1.7141839 ,  1.7146591 ],
         ...,
         [ 1.7201027 ,  1.7201834 ,  1.7202641 , ...,  1.7595494 ,
           1.7595775 ,  1.7596056 ],
         [ 1.7201512 ,  1.7202319 ,  1.7203125 , ...,  1.7595594 ,
           1.7595875 ,  1.7596157 ],
         [ 1.7201996 ,  1.7202803 ,  1.7203609 , ...,  1.7595694 ,
           1.7595974 ,  1.7596256 ]]]], dtype=float32)
Coordinates:
  * time         (time) datetime64[ns] 56B 2020-01-04T01:55:30.147982 ... 202...
  * tide_model   (tide_model) object 16B 'EOT20' 'GOT5.5'
  * y            (y) float64 5kB -1.952e+06 -1.952e+06 ... -1.971e+06 -1.971e+06
  * x            (x) float64 5kB -1.044e+06 -1.044e+06 ... -1.026e+06 -1.026e+06
    spatial_ref  int32 4B 3577

In [5]:
stats_ds.dims

FrozenMappingWarningOnValuesAccess({'x': 10, 'y': 9})

In [9]:
satellite_ds.x

<xarray.DataArray 'x' (x: 609)> Size: 5kB
array([-1044045., -1044015., -1043985., ..., -1025865., -1025835., -1025805.])
Coordinates:
  * x            (x) float64 5kB -1.044e+06 -1.044e+06 ... -1.026e+06 -1.026e+06
    spatial_ref  int32 4B 3577
Attributes:
    units:       metre
    resolution:  30.0
    crs:         EPSG:3577